# SMILE Twitter Dataset Sentiment Analysis with BERT 




Exploratory Data Analysis and Preprocessing 

In [4]:
import torch 
import pandas as pd 
from tqdm.notebook import tqdm 

In [5]:
df = pd.read_csv("/content/smile-annotations-final.csv", names=["ID", "Text", "Category"])
df.set_index("ID", inplace =True )

In [6]:
df.head()

,Text,Category
ID,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [7]:
df.Text.iloc[0]

'@aandraous @britishmuseum @AndrewsAntonio Merci pour le partage! @openwinemap'

Based on the categories of the dataset it seems that there are a mixture of emotions as well as non-emotions i.e nocode

I will drop nocode and the categories that contain 2 emotions as it will make training the BERT model easier for the purposes of this. 

In [8]:
df.Category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: Category, dtype: int64

In [9]:
df = df[((df.Category != 'nocode') &( df.Category != "happy|surprise") & (df.Category != 'happy|sad') & (df.Category != "disgust|angry") & (df.Category != "sad|angry") & (df.Category != "sad|disgust|angry"))]

In [10]:
df.Category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
sad|disgust        2
Name: Category, dtype: int64

In [11]:
# Turn categories into a dictionary of unique values
# Will be used in one-hot-encoding later 
labels = df.Category.unique()
label_dict = {}
for i, label in enumerate(labels):
  label_dict[label] = i  



In [12]:
label_dict

{'angry': 2,
 'disgust': 3,
 'happy': 0,
 'not-relevant': 1,
 'sad': 4,
 'sad|disgust': 6,
 'surprise': 5}

In [13]:
# Add the label_dict as a column in the dataframe 

df["Label"]= df.Category.replace(label_dict)
df.head()

,Text,Category,Label
ID,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0


# Training and Validation 
The data seems to have a problem where the distribution of it is skewed. 

There are 1137 examples of the category "happy" which is more than the combination of the rest of the other 6 categories combined!

Splitting the dataset into a train and validation set that is randomly shuffled is not going to cut it and will result in what is known as **sampling bias**. 

So instead try to split the categories according to their respective distribution in the dataset. 

This is known as Stratified Sampling. 



In [14]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, df.Label.values, test_size=0.2, random_state=42, stratify=df.Label.values)

In [15]:
# Create a column that shows how the data is split between the test and validation sets

df["data_type"] = ["not_set"]*df.shape[0]

In [16]:
df.head()

,Text,Category,Label,data_type
ID,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,not_set
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,not_set
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,not_set
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,not_set
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,not_set


In [17]:
df.loc[X_train, "data_type"] = "train"
df.loc[X_val, "data_type"] = "val"

In [18]:
df.head()

,Text,Category,Label,data_type
ID,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,train
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,val
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,train
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,train
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,train


In [19]:
# Groupby category, label, data_type 
df.groupby(["Category", "Label", "data_type"]).count()

Text
Category     Label data_type      
angry        2     train        45
                   val          12
disgust      3     train         5
                   val           1
happy        0     train       909
                   val         228
not-relevant 1     train       171
                   val          43
sad          4     train        26
                   val           6
sad|disgust  6     train         2
surprise     5     train        28
                   val           7

# Tokenizing and Encoding the data 

In [21]:
!pip install transformers

     |████████████████████████████████| 778kB 3.4MB/s 
     |████████████████████████████████| 1.1MB 16.9MB/s 
     |████████████████████████████████| 3.0MB 24.0MB/s 
     |████████████████████████████████| 890kB 42.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=e018eece031de980962cfd094c7451be5d637d70df9946a9cf33b4386d5de5a2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [22]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [23]:
# All the words in the dataset are lowercase so here just use bert-base-uncased
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True 
)


In [24]:
# Encode the words
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type == "train"].Text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length= 256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type == "val"].Text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length= 256,
    return_tensors='pt'
)

input_ids_train = encoded_data_train["input_ids"]
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type == "train"].Label.values)

input_ids_val = encoded_data_val["input_ids"]
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type == "val"].Label.values)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [25]:
# Data has been encoded time to split into into respective datasets

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [26]:
print("Number of items in training set :", len(dataset_train))

Number of items in training set : 1186


In [27]:
print("Number of items in validation set :", len(dataset_val))

Number of items in validation set : 297


# BERT pretrained model

In [28]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels= len(label_dict),
    output_attentions = False,
    output_hidden_states = False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# Creating Data Loaders 

In [29]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler



In [30]:
dataloader_train = DataLoader(
    dataset_train, 
    batch_size = 32,
    sampler = RandomSampler(dataset_train)
)

dataloader_val = DataLoader(
    dataset_val, 
    batch_size = 32,
    sampler = RandomSampler(dataset_val)
)

# Optimizer and Scheduler 

In [31]:
from transformers import AdamW, get_linear_schedule_with_warmup


In [49]:
optimizer = AdamW(
    model.parameters(),
    lr=2e-5,
    eps=1e-8
)

In [51]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = len(dataloader_train)*epochs
)

# Performance Metrics 

In [52]:
import numpy as np
from sklearn.metrics import f1_score

In [53]:
# F1 score 
# Turn predictions from a vector of probabilities to a vector where the highest prob is 1 and the rest are 0
def f1_score_(preds, labels):
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return f1_score(labels_flat, preds_flat, average="weighted")

In [54]:
def accuracy(preds, labels):
  label_inverse = {v:k for k,v in label_dict.items()}
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()

  for label in np.unique(labels_flat):
    y_preds = preds_flat[labels_flat == label]
    y_true = labels_flat[labels_flat == label]
    print(f'Class: {label_inverse[label]}')
    print(f'Accuracy: {len(y_preds[y_preds == label])}/{len(y_true)}\n')

# Training Loop

In [55]:
import random 

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [56]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [65]:
def evaluate(dataloader_val):
  model.eval()

  loss_val_total = 0
  predictions, true_vals = [], []
  for batch in tqdm(dataloader_val):
    batch = tuple(b.to(device) for b in batch)
    inputs = {
        'input_ids': batch[0],
        'attention_mask': batch[1],
        'labels': batch[2]
    }
    with torch.no_grad():
      outputs = model(**inputs)

    loss = outputs[0]
    logits = outputs[1]
    loss_val_total += loss.item()

    logits = logits.detach().cpu().numpy()
    label_ids = inputs['labels'].cpu().numpy()
    predictions.append(logits)
    true_vals.append(label_ids)
  loss_val_avg = loss_val_total/len(dataloader_val)
  predictions = np.concatenate(predictions, axis=0)
  true_vals = np.concatenate(true_vals, axis=0)
  return loss_val_avg, predictions, true_vals

In [62]:
for epoch in tqdm(range(1, epochs+1)):
  model.train()
  loss_train_total = 0
  progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch),
                      leave= False,
                      disable=False)
  for batch in progress_bar:
    model.zero_grad()
    batch = tuple(b.to(device) for b in batch)
    inputs = {
        'input_ids': batch[0],
        'attention_mask': batch[1],
        'labels': batch[2]
    }
    outputs = model(**inputs)
    loss = outputs[0]
    loss_train_total += loss.item()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    optimizer.step()
    scheduler.step()
    progress_bar.set_postfix({'Training loss': '{:.3f}'.format(loss.item()/len(batch))})
  torch.save(model.state_dict(), f'/content/Models/BERT_ft_epoch{epoch}.model')
  tqdm.write(f'\nEpoch {epoch}')
  loss_train_avg = loss_train_total / len(dataloader_train)
  tqdm.write(f'Training loss: {loss_train_avg}')
  val_loss, predictions, true_vals = evaluate(dataloader_val)
  val_f1 = f1_score_(predictions, true_vals)
  tqdm.write(f'Validation loss: {val_loss}')
  tqdm.write(f'F1 score (weighted): {val_f1}')



Epoch 1
Training loss: 0.10521472627787214
Validation loss: 0.5429523140192032
F1 score (weighted): 0.8778144128235467



Epoch 2
Training loss: 0.0661352134652828
Validation loss: 0.6351218819618225
F1 score (weighted): 0.8743908929899904



Epoch 3
Training loss: 0.047395023860429465
Validation loss: 0.5871795699000358
F1 score (weighted): 0.8714892817910667



Epoch 4
Training loss: 0.03375027887523174
Validation loss: 0.5735796310007573
F1 score (weighted): 0.8729187149987443



Epoch 5
Training loss: 0.025298164275131728
Validation loss: 0.6933273404836655
F1 score (weighted): 0.8860599803689234



Epoch 6
Training loss: 0.02173128861345743
Validation loss: 0.5890041818842292
F1 score (weighted): 0.8681413017522054



Epoch 7
Training loss: 0.019776597128886925
Validation loss: 0.6088147991336882
F1 score (weighted): 0.870480992151047



Epoch 8
Training loss: 0.01906392468433631
Validation loss: 0.735888497531414
F1 score (weighted): 0.870480992151047



Epoch 9
Training loss: 0.017981956467816706
Validation loss: 0.6885843992233276
F1 score (weighted): 0.870480992151047



Epoch 10
Training loss: 0.01912587430132063
Validation loss: 0.6677531719207763
F1 score (weighted): 0.870480992151047



In [63]:
model.load_state_dict(
    torch.load('/content/Models/BERT_ft_epoch1.model',
               map_location=torch.device('cpu'))
)

<All keys matched successfully>

In [66]:
_, predictions, true_vals = evaluate(dataloader_val)

In [68]:
accuracy(predictions, true_vals)

Class: happy
Accuracy: 211/228

Class: not-relevant
Accuracy: 32/43

Class: angry
Accuracy: 10/12

Class: disgust
Accuracy: 0/1

Class: sad
Accuracy: 3/6

Class: surprise
Accuracy: 4/7

